In [0]:
# %sql
# select 
#   date_ym, count(1)
# from kic_data_mart.tv_usage.app_usage_by_user
# where 1=1
#   and platform_version = 'webOS25'
#   and country_code ='KR'
#   and date_ym >='2025-01'
# group by 
#   date_ym
# order by 
#   date_ym

In [0]:
# %sql
# select 
#   date_ym, count(1)
# from kic_data_mart.tv_usage.iot_usage
# where 1=1
#   and platform_version = 'webOS25'
#   and country_code ='KR'
#   and date_ym >='2025-01'
# group by 
#   date_ym
# order by 
#   date_ym

In [0]:
# %sql
# select *
# from kic_data_ods.lgcmp.channel
# where 1=1
#   and webosVersion like '%webOSTV 25%'
#   and date_ym = '2025-06'
#   and country ='KR'

In [0]:
# %sql
# select 
#   date_ym, count(1), count(distinct macAddress)
# from kic_data_ods.lgcmp.channel
# where 1=1
#   and webosVersion like '%webOSTV 25%'
#   and date_ym >= '2025-06'
# group by 
#   date_ym

In [0]:
# iot_mac_df = spark.sql("""
#     select distinct mac_addr
#     from kic_data_mart.tv_usage.iot_usage
#     where platform_version = 'webOS25'
#       and country_code = 'KR'
#       and date_ym = '2025-06'
# """)

# app_mac_df = spark.sql("""
#     select distinct mac_addr
#     from kic_data_mart.tv_usage.app_usage_by_user
#     where platform_version = 'webOS25'
#       and country_code = 'KR'
#       and date_ym = '2025-06'
# """)

# remote_mac_df = spark.sql("""
#     select distinct mac_addr
#     from kic_data_mart.tv_usage.remote_controller_by_user
#     where platform_version = 'webOS25'
#       and date_ym = '2025-06'
# """)

# iot_mac_count = iot_mac_df.count()
# app_mac_in_iot_count = iot_mac_df.join(app_mac_df, iot_mac_df.mac_addr == app_mac_df.mac_addr, "inner").count()
# remote_mac_in_iot_count = iot_mac_df.join(remote_mac_df, iot_mac_df.mac_addr == remote_mac_df.mac_addr, "inner").count()

# app_mac_ratio = app_mac_in_iot_count / iot_mac_count if iot_mac_count > 0 else None
# remote_mac_ratio = remote_mac_in_iot_count / iot_mac_count if iot_mac_count > 0 else None

# display(
#   spark.createDataFrame(
#     [
#       ("iot_usage", 1.0, iot_mac_count),
#       ("app_usage_by_user", app_mac_ratio, app_mac_in_iot_count),
#       ("remote_controller_by_user", remote_mac_ratio, remote_mac_in_iot_count )
#     ],
#     ["table", "mac_ratio","mac_cnt"]
#   )
# )

In [0]:
# # 기존 3개 소스 (변경 없음)
# iot_mac_df = spark.sql("""
#     select distinct mac_addr
#     from kic_data_mart.tv_usage.iot_usage
#     where platform_version = 'webOS25'
#       and country_code = 'KR'
#       and date_ym = '2025-06'
# """)

# app_mac_df = spark.sql("""
#     select distinct mac_addr
#     from kic_data_mart.tv_usage.app_usage_by_user
#     where platform_version = 'webOS25'
#       and country_code = 'KR'
#       and date_ym = '2025-06'
# """)

# remote_mac_df = spark.sql("""
#     select distinct mac_addr
#     from kic_data_mart.tv_usage.remote_controller_by_user
#     where platform_version = 'webOS25'
#       and date_ym = '2025-06'
# """)

# # 신규: ODS 채널/프로그램 (macAddress -> mac_addr 로 alias)
# channel_mac_df = spark.sql("""
#     select distinct macAddress as mac_addr
#     from kic_data_ods.lgcmp.channel
#     where webosVersion like '%webOSTV 25%'
#       and date_ym = '2025-06'
#       and country = 'KR'
# """)

# program_mac_df = spark.sql("""
#     select distinct macAddress as mac_addr
#     from kic_data_ods.lgcmp.program
#     where webosVersion like '%webOSTV 25%'
#       and date_ym = '2025-06'
#       and country = 'KR'
# """)

# # 기준母集: iot
# iot_mac_count = iot_mac_df.count()

# # 교집합 카운트
# app_mac_in_iot_count     = iot_mac_df.join(app_mac_df,     "mac_addr", "inner").count()
# remote_mac_in_iot_count  = iot_mac_df.join(remote_mac_df,  "mac_addr", "inner").count()
# channel_mac_in_iot_count = iot_mac_df.join(channel_mac_df, "mac_addr", "inner").count()
# program_mac_in_iot_count = iot_mac_df.join(program_mac_df, "mac_addr", "inner").count()

# # 비율
# app_mac_ratio     = app_mac_in_iot_count     / iot_mac_count if iot_mac_count > 0 else None
# remote_mac_ratio  = remote_mac_in_iot_count  / iot_mac_count if iot_mac_count > 0 else None
# channel_mac_ratio = channel_mac_in_iot_count / iot_mac_count if iot_mac_count > 0 else None
# program_mac_ratio = program_mac_in_iot_count / iot_mac_count if iot_mac_count > 0 else None

# # 보기 좋게 테이블로 출력
# result_df = spark.createDataFrame(
#     [
#       ("iot_usage (base)",               1.0,                iot_mac_count),
#       ("app_usage_by_user",              app_mac_ratio,      app_mac_in_iot_count),
#       ("remote_controller_by_user",      remote_mac_ratio,   remote_mac_in_iot_count),
#       ("lgcmp.channel (ODS)",            channel_mac_ratio,  channel_mac_in_iot_count),
#       ("lgcmp.program (ODS)",            program_mac_ratio,  program_mac_in_iot_count),
#     ],
#     ["table", "mac_ratio_vs_iot", "mac_cnt_in_iot"]
# )

# display(result_df)

In [0]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# -------------------------------------------
# 1) 조회할 월 리스트 생성: 2025-06 ~ 2025-12
# -------------------------------------------
start = datetime(2025, 6, 1)
end   = datetime(2025, 12, 1)

months = []
cur = start
while cur <= end:
    months.append(cur.strftime("%Y-%m"))
    cur += relativedelta(months=1)

months

In [0]:
# -------------------------------------------
# 2) 소스별 MAC 추출 함수
#    (필요 시 MAC 정규화 주석 참고)
# -------------------------------------------

def get_iot_mac_df(month_ym: str):
    return spark.sql(f"""
        select distinct mac_addr
        from kic_data_mart.tv_usage.iot_usage
        where platform_version = 'webOS25'
          and country_code = 'KR'
          and date_ym = '{month_ym}'
    """)
    # 정규화 예시:
    # return spark.sql(f"""
    #     select distinct lower(regexp_replace(mac_addr, '[:\\-]', '')) as mac_addr
    #     from kic_data_mart.tv_usage.iot_usage
    #     where platform_version = 'webOS25'
    #       and country_code = 'KR'
    #       and date_ym = '{month_ym}'
    # """)

def get_app_mac_df(month_ym: str):
    return spark.sql(f"""
        select distinct mac_addr
        from kic_data_mart.tv_usage.app_usage_by_user
        where platform_version = 'webOS25'
          and country_code = 'KR'
          and date_ym = '{month_ym}'
    """)

def get_remote_mac_df(month_ym: str):
    return spark.sql(f"""
        select distinct mac_addr
        from kic_data_mart.tv_usage.remote_controller_by_user
        where platform_version = 'webOS25'
          and date_ym = '{month_ym}'
    """)

def get_channel_mac_df(month_ym: str):
    return spark.sql(f"""
        select distinct macAddress as mac_addr
        from kic_data_ods.lgcmp.channel
        where webosVersion like '%webOSTV 25%'
          and date_ym = '{month_ym}'
          and country = 'KR'
    """)

def get_program_mac_df(month_ym: str):
    return spark.sql(f"""
        select distinct macAddress as mac_addr
        from kic_data_ods.lgcmp.program
        where webosVersion like '%webOSTV 25%'
          and date_ym = '{month_ym}'
          and country = 'KR'
    """)
    

In [0]:
# -------------------------------------------
# 3) 월별 계산 루프
#    - iot을 모수로 하여 각 소스 교집합(비율/카운트) 계산
#    - + 5개 공통 MAC 개수/비율(ALL_5_COMMON) 추가
# -------------------------------------------

rows = []

for ym in months:
    iot_df     = get_iot_mac_df(ym).cache()
    app_df     = get_app_mac_df(ym)
    remote_df  = get_remote_mac_df(ym)
    channel_df = get_channel_mac_df(ym)
    program_df = get_program_mac_df(ym)

    iot_cnt = iot_df.count()

    # 개별 교집합: iot 기준
    app_in_iot_cnt     = iot_df.join(app_df,     "mac_addr", "inner").count()
    remote_in_iot_cnt  = iot_df.join(remote_df,  "mac_addr", "inner").count()
    channel_in_iot_cnt = iot_df.join(channel_df, "mac_addr", "inner").count()
    program_in_iot_cnt = iot_df.join(program_df, "mac_addr", "inner").count()

    app_ratio     = (app_in_iot_cnt     / iot_cnt) if iot_cnt > 0 else None
    remote_ratio  = (remote_in_iot_cnt  / iot_cnt) if iot_cnt > 0 else None
    channel_ratio = (channel_in_iot_cnt / iot_cnt) if iot_cnt > 0 else None
    program_ratio = (program_in_iot_cnt / iot_cnt) if iot_cnt > 0 else None

    # 5개 공통 MAC: iot을 기준으로 left_semi로 단계적 필터
    common_5_cnt = (
        iot_df
        .join(app_df,     "mac_addr", "left_semi")
        .join(remote_df,  "mac_addr", "left_semi")
        .join(channel_df, "mac_addr", "left_semi")
        .join(program_df, "mac_addr", "left_semi")
        .count()
    )
    common_5_ratio = (common_5_cnt / iot_cnt) if iot_cnt > 0 else None

    # 결과 행 쌓기 (iot 모수도 함께 기록)
    rows.extend([
        (ym, "iot_usage (base)",              1.0,               iot_cnt,            iot_cnt),
        (ym, "app_usage_by_user",             app_ratio,         app_in_iot_cnt,     iot_cnt),
        (ym, "remote_controller_by_user",     remote_ratio,      remote_in_iot_cnt,  iot_cnt),
        (ym, "lgcmp.channel (ODS)",           channel_ratio,     channel_in_iot_cnt, iot_cnt),
        (ym, "lgcmp.program (ODS)",           program_ratio,     program_in_iot_cnt, iot_cnt),
        (ym, "ALL_5_COMMON",                  common_5_ratio,    common_5_cnt,       iot_cnt),  # << 추가
    ])

    iot_df.unpersist()

result_df = spark.createDataFrame(
    rows,
    schema=["date_ym", "table", "mac_ratio_vs_iot", "mac_cnt_in_iot", "iot_mac_cnt"]
)

display(result_df.orderBy("date_ym", "table"))

In [0]:
from pyspark.sql import DataFrame

# ============================================================
# 0) 테이블명 설정 (환경에 맞게 반드시 치환)
# ============================================================
APP_TBL     = "kic_data_mart.tv_usage.app_usage_by_user"           
IOT_TBL     = "kic_data_mart.tv_usage.iot_usage"           
REMOTE_TBL  = "kic_data_mart.tv_usage.remote_controller_by_user"   
CHANNEL_TBL = "kic_data_ods.lgcmp.channel"                        
PROGRAM_TBL = "kic_data_ods.lgcmp.program"                        

# 공통 필터 값
PLATFORM_VERSION = "webOS25"
WEBOS_VERSION = "webOSTV 25"
COUNTRY_CODE = "KR"

# ============================================================
# 1) MAC만 가져오는 함수들 (distinct mac_addr)
# ============================================================
def get_app_mac_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select distinct mac_addr
        from {APP_TBL}
        where platform_version = '{PLATFORM_VERSION}'
          and country_code = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_iot_mac_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select distinct mac_addr
        from {IOT_TBL}
        where platform_version = '{PLATFORM_VERSION}'
          and country_code = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_remote_mac_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select distinct mac_addr
        from {REMOTE_TBL}
        where platform_version = '{PLATFORM_VERSION}'
          and country_code = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_channel_mac_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select distinct macAddress as mac_addr
        from {CHANNEL_TBL}
        where webosVersion like '{WEBOS_VERSION}'
          and country = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_program_mac_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select distinct macAddress as mac_addr
        from {PROGRAM_TBL}
        where webosVersion like '{WEBOS_VERSION}'
          and country = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)


# ============================================================
# 2) 원본 "전체 스키마" 가져오는 함수들 (select *)
#    - 500개 MAC 필터링 결과를 "동일 스키마"로 만들기 위해 필요
# ============================================================
def get_app_full_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select *
        from {APP_TBL}
        where platform_version = '{PLATFORM_VERSION}'
          and country_code = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_iot_full_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select *
        from {IOT_TBL}
        where platform_version = '{PLATFORM_VERSION}'
          and country_code = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_remote_full_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select *
        from {REMOTE_TBL}
        where platform_version = '{PLATFORM_VERSION}'
          and country_code = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_channel_full_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select *, macAddress as mac_addr
        from {CHANNEL_TBL}
        where webosVersion like '{WEBOS_VERSION}'
          and country = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)

def get_program_full_df(month_ym: str) -> DataFrame:
    return spark.sql(f"""
        select *, macAddress as mac_addr
        from {PROGRAM_TBL}
        where webosVersion like '{WEBOS_VERSION}'
          and country = '{COUNTRY_CODE}'
          and date_ym = '{month_ym}'
    """)


# ============================================================
# 3) (옵션) 테이블명/컬럼 검증 유틸
#    - 실제 테이블명이 맞는지 빠르게 확인할 때 사용
# ============================================================
def assert_table_has_columns(table_name: str, required_cols: list[str]):
    cols = set(spark.table(table_name).columns)
    missing = [c for c in required_cols if c not in cols]
    if missing:
        raise ValueError(f"[{table_name}] missing columns: {missing}")

# 예) 최소 mac_addr/date_ym 존재 확인
# assert_table_has_columns(APP_TBL, ["mac_addr", "date_ym", "platform_version", "country_code"])


In [0]:
from pyspark.sql import functions as F
from functools import reduce


TARGET_MAC_TBL = "adhoc.heds.1256_targer_mac"

OUT_IOT_TBL     = "adhoc.heds.1256_iot_usage_target500"
OUT_APP_TBL     = "adhoc.heds.1256_app_usage_by_user_target500"
OUT_REMOTE_TBL  = "adhoc.heds.1256_remote_controller_by_user_target500"
OUT_CHANNEL_TBL = "adhoc.heds.1256_lgcmp_channel_ods_target500"
OUT_PROGRAM_TBL = "adhoc.heds.1256_lgcmp_program_ods_target500"

SEED = 1256
N_TARGET = 500

# (권장) 테이블 생성/저장 시 date_ym 파티션 사용 가능하면 사용
# 아래 write 파트에서 partitionBy("date_ym") 적용

def build_month_common_5_mac_500(ym: str):
    """월(ym)별 5개 공통 MAC에서 최대 500개 샘플링하여 (date_ym, mac_addr) 반환"""
    iot_df     = get_iot_mac_df(ym).select("mac_addr").dropDuplicates()
    app_df     = get_app_mac_df(ym).select("mac_addr").dropDuplicates()
    remote_df  = get_remote_mac_df(ym).select("mac_addr").dropDuplicates()
    channel_df = get_channel_mac_df(ym).select("mac_addr").dropDuplicates()
    program_df = get_program_mac_df(ym).select("mac_addr").dropDuplicates()

    common_5_df = (
        iot_df
        .join(app_df,     "mac_addr", "left_semi")
        .join(remote_df,  "mac_addr", "left_semi")
        .join(channel_df, "mac_addr", "left_semi")
        .join(program_df, "mac_addr", "left_semi")
        .dropDuplicates(["mac_addr"])
    )

    # 월별 최대 500개 샘플 (결정적 샘플링을 원하면 seed 고정)
    sampled_df = (
        common_5_df
        .orderBy(F.rand(SEED))
        .limit(N_TARGET)
        .withColumn("date_ym", F.lit(ym))
        .select("date_ym", "mac_addr")
    )
    return sampled_df

# -----------------------------
# 1) 월별 공통 MAC 500개를 target 테이블에 저장
# -----------------------------
target_mac_all = None
for ym in months:
    mdf = build_month_common_5_mac_500(ym)
    target_mac_all = mdf if target_mac_all is None else target_mac_all.unionByName(mdf)

# overwrite로 재생성 (원하면 append로 변경)
(
    target_mac_all
    .write
    .mode("overwrite")
    .format("delta")
    .partitionBy("date_ym")
    .saveAsTable(TARGET_MAC_TBL)
)

# -----------------------------
# 2) 각 테이블에서 target 500개 MAC만 필터링하여 "원본과 동일 스키마"로 저장
#    - left_semi join을 사용하면 원본 컬럼을 그대로 유지합니다.
# -----------------------------

# target MAC를 전체 로딩 (월별 필터는 date_ym로)
target_mac_tbl_df = spark.table(TARGET_MAC_TBL).select("date_ym", "mac_addr")

def filter_full_by_target_mac(full_df, ym: str):
    """full_df(원본 전체 스키마)에서 해당 월의 target 500 mac만 남김"""
    ym_target = (
        target_mac_tbl_df
        .filter(F.col("date_ym") == ym)
        .select("mac_addr")
        .dropDuplicates()
    )
    # mac 기준 left_semi -> full_df 스키마 그대로 유지
    return full_df.join(F.broadcast(ym_target), "mac_addr", "left_semi")

# 아래 get_*_full_df(ym)는 사용 환경에 맞게 준비해야 합니다.
# (예) spark.table("db.tbl").where(col("date_ym")==ym) 같은 형태

iot_out_all = None
app_out_all = None
remote_out_all = None
channel_out_all = None
program_out_all = None

for ym in months:
    # ---- 원본 전체 스키마 DF 로딩 (사용자 환경에 맞게 구현 필요) ----
    iot_full_df     = get_iot_full_df(ym)
    app_full_df     = get_app_full_df(ym)
    remote_full_df  = get_remote_full_df(ym)
    channel_full_df = get_channel_full_df(ym)
    program_full_df = get_program_full_df(ym)

    # (선택) 원본에 date_ym이 없다면 추가
    # iot_full_df = iot_full_df.withColumn("date_ym", F.lit(ym))  # 필요 시만
    # ...

    iot_f     = filter_full_by_target_mac(iot_full_df, ym)
    app_f     = filter_full_by_target_mac(app_full_df, ym)
    remote_f  = filter_full_by_target_mac(remote_full_df, ym)
    channel_f = filter_full_by_target_mac(channel_full_df, ym)
    program_f = filter_full_by_target_mac(program_full_df, ym)

    iot_out_all     = iot_f     if iot_out_all     is None else iot_out_all.unionByName(iot_f, allowMissingColumns=False)
    app_out_all     = app_f     if app_out_all     is None else app_out_all.unionByName(app_f, allowMissingColumns=False)
    remote_out_all  = remote_f  if remote_out_all  is None else remote_out_all.unionByName(remote_f, allowMissingColumns=False)
    channel_out_all = channel_f if channel_out_all is None else channel_out_all.unionByName(channel_f, allowMissingColumns=False)
    program_out_all = program_f if program_out_all is None else program_out_all.unionByName(program_f, allowMissingColumns=False)

# 테이블 저장 (overwrite / delta / 월 파티션 권장)
(
    iot_out_all.write.mode("overwrite").format("delta").saveAsTable(OUT_IOT_TBL)
)
(
    app_out_all.write.mode("overwrite").format("delta").saveAsTable(OUT_APP_TBL)
)
(
    remote_out_all.write.mode("overwrite").format("delta").saveAsTable(OUT_REMOTE_TBL)
)
(
    channel_out_all.write.mode("overwrite").format("delta").saveAsTable(OUT_CHANNEL_TBL)
)
(
    program_out_all.write.mode("overwrite").format("delta").saveAsTable(OUT_PROGRAM_TBL)
)


In [0]:
iot_df     = get_iot_mac_df(ym).select("mac_addr").dropDuplicates()
app_df     = get_app_mac_df(ym).select("mac_addr").dropDuplicates()
remote_df  = get_remote_mac_df(ym).select("mac_addr").dropDuplicates()
channel_df = get_channel_mac_df(ym).select("mac_addr").dropDuplicates()
program_df = get_program_mac_df(ym).select("mac_addr").dropDuplicates()

common_5_df = (
    iot_df
    .join(app_df,     "mac_addr", "left_semi")
    .join(remote_df,  "mac_addr", "left_semi")
    .join(channel_df, "mac_addr", "left_semi")
    .join(program_df, "mac_addr", "left_semi")
    .dropDuplicates(["mac_addr"])
)

# 월별 최대 500개 샘플 (결정적 샘플링을 원하면 seed 고정)
sampled_df = (
    common_5_df
    .orderBy(F.rand(SEED))
    .limit(N_TARGET)
    .withColumn("date_ym", F.lit(ym))
    .select("date_ym", "mac_addr")
)

In [0]:
%sql
select 
  date_ym, count(1)
from adhoc.heds.`1256_targer_mac`
group by date_ym

In [0]:
%sql
select 
  date_ym, count(1), count(distinct mac_addr)
from adhoc.heds.`1256_app_usage_by_user_target500`
group by date_ym

In [0]:
%sql
select 
  date_ym, count(1), count(distinct mac_addr)
from adhoc.heds.`1256_lgcmp_channel_ods_target500`
group by date_ym

In [0]:
%sql
select 
  date_ym, count(1), count(distinct mac_addr)
from adhoc.heds.`1256_lgcmp_program_ods_target500`
group by date_ym

In [0]:
%sql
select *
from adhoc.heds.`1256_lgcmp_program_ods_target500`

In [0]:
%sql
select *
from adhoc.heds.`1256_lgcmp_channel_ods_target500`